In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt 
import time 
import numpy as np 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MinMaxScaler 
from pylab import mpl 
from sklearn.preprocessing import StandardScaler 
import seaborn as sns 
import torch.nn as nn 
import torch 
from torch.utils.data import Dataset,DataLoader 
from torch.autograd import Variable
import torch.utils.data as Data 
mpl.rcParams['font.sans-serif'] = ['SimHei']
mpl.rcParams['axes.unicode_minus'] = False
from sklearn import metrics 
import matplotlib  
import warnings
warnings.filterwarnings('ignore')
df=pd.read_csv('continuous dataset.csv') 
df['datetime']=pd.to_datetime(df['datetime'])
df=df.set_index('datetime') 
df 

In [ ]:
#缺失值
df.isnull().sum()

异常值处理

In [ ]:
plt.figure(figsize=(12, 4),dpi=120)
sns.lineplot(x=df.index,y=df['nat_demand'])
plt.xticks(rotation=30, fontsize=17)# 横坐标倾斜
plt.xlabel('date', fontsize=19)
plt.yticks(fontsize=17)
plt.ylabel('nat_demand', fontsize=19) 


#plt.xticks(rotation=30, fontsize=10)  
#plt.yticks(fontsize=10) 
#plt.xlabel('Date', fontsize=12)  
#plt.ylabel('Demand', fontsize=12)  
#plt.title('Natural Gas Demand', fontsize=14) 



# plt.savefig('nat_demand.jpg',bbox_inches = 'tight',dpi=600) 
plt.show() 


绘制箱线图

In [ ]:
#去除异常值前
plt.figure(figsize=(12, 4),dpi=120) # 大小为宽度12英寸，高度4英寸，分辨率为120。
sns.boxplot(y=df['nat_demand']) 

In [ ]:
Ser=df['nat_demand'] 
Low = Ser.quantile(0.25)-1.5*(Ser.quantile(0.75)-Ser.quantile(0.25)) 
Up = Ser.quantile(0.75)+1.5*(Ser.quantile(0.75)-Ser.quantile(0.25)) 
print(Low,Up) 

In [ ]:
df['nat_demand']=df['nat_demand'].map(lambda x:Up if x>Up else(Low if x<Low else x))

In [ ]:
#去除异常值后
plt.figure(figsize=(12, 4),dpi=120) # 大小为宽度12英寸，高度4英寸，分辨率为120。
sns.boxplot(y=df['nat_demand'])

特征筛选

In [ ]:
plt.rcParams['figure.dpi'] = 120 
fig = plt.gcf() 
fig.set_size_inches(15,10)  
ax=sns.heatmap(data=df.corr(),
            vmax=0.9,  
            # cmap=palettable.cmocean.diverging.Curl_10.mpl_colors,
            annot=True,  
            fmt=".2f",  
            # annot_kws={'size':8,'weight':'normal', 'color':'#253D24'},
            annot_kws={'size':16,'weight':'normal', 'color':'#253D24'},
            # mask=np.triu(np.ones_like(data[tmp_list].corr(),dtype=np.bool))
            
           )
#xticklabels = heatmap.get_xticklabels()
#yticklabels = heatmap.get_yticklabels()
#heatmap.set_xticklabels(xticklabels, fontsize=22)
#heatmap.set_yticklabels(yticklabels, fontsize=22)
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)

cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=22)


plt.show() 

In [ ]:
abs(df.corr()['nat_demand']).sort_values()

In [ ]:
#删除相关性低的变量QV2M_dav
df.drop(columns=['QV2M_dav'],axis=1,inplace=True)

#person特征选择前5
df.drop(columns=['QV2M_san'],axis=1,inplace=True)
df.drop(columns=['QV2M_toc'],axis=1,inplace=True)
df.drop(columns=['school'],axis=1,inplace=True)
df.drop(columns=['TQL_dav'],axis=1,inplace=True)
df.drop(columns=['W2M_dav'],axis=1,inplace=True)
df.drop(columns=['TQL_toc'],axis=1,inplace=True)
df.drop(columns=['W2M_toc'],axis=1,inplace=True)
df.drop(columns=['TQL_san'],axis=1,inplace=True)
df.drop(columns=['Holiday_ID'],axis=1,inplace=True)
df.drop(columns=['holiday'],axis=1,inplace=True)




### shap特征选择前5
#df.drop(columns=['school'],axis=1,inplace=True)
#df.drop(columns=['holiday'],axis=1,inplace=True)
#df.drop(columns=['Holiday_ID'],axis=1,inplace=True)
#df.drop(columns=['W2M_toc'],axis=1,inplace=True)
#df.drop(columns=['TQL_san'],axis=1,inplace=True)
#df.drop(columns=['T2M_toc'],axis=1,inplace=True)
#df.drop(columns=['W2M_san'],axis=1,inplace=True)
#df.drop(columns=['QV2M_toc'],axis=1,inplace=True)
#df.drop(columns=['W2M_dav'],axis=1,inplace=True)
#df.drop(columns=['TQL_toc'],axis=1,inplace=True)



数据处理

In [ ]:
def load_data(data, time_step=2, after_day=1, validate_percent=0.8): 
    seq_length = time_step + after_day 
    result = []
    index_list=[]

    for index in range(len(data) - seq_length + 1): 
        result.append(data[index: index + seq_length])
        index_list.append(np.array(data.index)[index: index + seq_length])
    result = np.array(result)
    index_list = np.array(index_list)
    print('total data \n', result.shape)
    train_size = int(len(result) * validate_percent) 
    train = result[:train_size, :] 
    validate = result[train_size:, :] 

    x_train = train[:, :time_step]
    y_train = train[:, time_step:][:,:,0]
    x_validate = validate[:, :time_step]
    y_validate = validate[:, time_step:][:,:,0]#取第一列
    
    #取index
    train_index=index_list[:train_size, :]
    validate_index=index_list[train_size:, :]
    x_train_index = train_index[:, :time_step]
    y_train_index = train_index[:,time_step:]
    x_validate_index = validate_index[:, :time_step]
    y_validate_index = validate_index[:, time_step:]
   
    scaler1 = MinMaxScaler(feature_range=(0, 1)) 
    x_train=scaler1.fit_transform(x_train.reshape(x_train.shape[0],x_train.shape[1]*x_train.shape[2])).reshape(x_train.shape[0],x_train.shape[1],x_train.shape[2])
    x_validate=scaler1.transform(x_validate.reshape(x_validate.shape[0],x_validate.shape[1]*x_validate.shape[2])).reshape(x_validate.shape[0],x_validate.shape[1],x_validate.shape[2])
    scaler2 = MinMaxScaler(feature_range=(0, 1))
    y_train=scaler2.fit_transform(y_train)
    y_validate=scaler2.transform(y_validate)
    return [scaler1,scaler2,result,x_train, y_train, x_validate, y_validate,y_train_index,y_validate_index]
time_step=7
after_day=1
scaler1,scaler2,data,x_train, y_train, x_test, y_test,y_train_index,y_validate_index = load_data(df,time_step=time_step, after_day=after_day, validate_percent=0.8)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)



In [ ]:
# x_train=x_train[:int(x_train.shape[0]/30)*30]
# y_train=y_train[:int(y_train.shape[0]/30)*30]
# x_test=x_test[:int(x_test.shape[0]/30)*30]
# y_test=y_test[:int(y_test.shape[0]/30)*30]


In [ ]:
x_train=Variable(torch.Tensor(x_train))
x_test=Variable(torch.Tensor(x_test))
y_train=Variable(torch.Tensor(y_train))
y_test=Variable(torch.Tensor(y_test))



In [ ]:
# 数据装入
def data_generator(x_train,x_test,y_train,y_test,batch_size):
    train_dataset=Data.TensorDataset(x_train,y_train)
    test_dataset=Data.TensorDataset(x_test,y_test)
    train_loader=torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=False,drop_last=True) 
    test_loader=torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False,drop_last=True)
    return train_loader,test_loader

In [ ]:
train_loader,test_loader=data_generator(x_train,x_test,y_train,y_test,30)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

定义模型

In [ ]:
class BILSTM_Attention(nn.Module):
    def __init__(self,input_size,hidden_size,output_size,num_layers,batch_size):
        super().__init__() 
        self.input_size=input_size 
        self.hidden_size=hidden_size 
        self.num_layers=num_layers 
        self.batch_size=batch_size 
        self.output_size=output_size 
        model_name=='BILSTM-ATT':        
        self.num_directions = 2 
        self.lstm=nn.LSTM(input_size ,hidden_size,num_layers,batch_first=False, bidirectional=True)
       
             
        self.linear1=nn.Linear(self.hidden_size*self.num_directions,self.hidden_size) 
        self.linear2=nn.Linear(self.hidden_size,output_size) 
        self.dropout=nn.Dropout(0.5)


    def attention_net(self, lstm_output, final_state): 

        lstm_output = lstm_output.permute(1, 0, 2)
        hidden = final_state.view(-1,self.num_directions*self.hidden_size,self.num_layers)  
        # print(lstm_output.shape,hidden.shape)
        self.attn_weights = torch.bmm(lstm_output, hidden) # attn_weights : [batch_size, n_step]

        soft_attn_weights = torch.tanh(self.attn_weights)

        context = torch.bmm(lstm_output.transpose(1, 2), soft_attn_weights).squeeze(2)
        return context # context : [batch_size, n_hidden * num_directions(=2)]

        
    def forward(self,input_seq):
        input_seq=input_seq.permute(1,0,2)
        h_0 = torch.randn(self.num_directions * self.num_layers, self.batch_size, self.hidden_size).to(device)
        c_0 = torch.randn(self.num_directions * self.num_layers, self.batch_size, self.hidden_size).to(device)       
        model_name=='BILSTM-ATT': 
        lstm_out, self.hidden_cell = self.lstm(input_seq)
        # print(lstm_out.shape,self.hidden_cell[0].shape)
        attn_output = self.attention_net(lstm_out,self.hidden_cell[0])
        # print(attn_output.shape)
      
        # print(attn_output.view(-1,self.hidden_size*self.num_directions).shape)
        out=self.linear1(attn_output.view(-1,self.hidden_size*self.num_directions))
        out=torch.tanh(out)
        # print(out.shape)
        predictions = self.linear2(out)

        return predictions
    


In [ ]:


class BiLSTM_BIGRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, batch_size):
        super().__init__()
        self.input_size = input_size 
        self.hidden_size = hidden_size 
        self.num_layers = num_layers 
        self.output_size = output_size 
        self.batch_size = batch_size 
        model_name=='BILSTM':
        self.num_directions = 2
        self.lstm = nn.LSTM(self.input_size, self.hidden_size, self.num_layers, batch_first=True, bidirectional=True)
        self.linear = nn.Linear(self.num_directions * self.hidden_size, self.output_size)
       
            
    def forward(self, input_seq):
        h_0 = torch.randn(self.num_directions * self.num_layers, self.batch_size, self.hidden_size).to(device)
        c_0 = torch.randn(self.num_directions * self.num_layers, self.batch_size, self.hidden_size).to(device)
        # print(input_seq.size())
        seq_len = input_seq.shape[1] # seq_len = input_seq.shape[1]
        model_name=='BILSTM':        
        output, _ = self.lstm(input_seq, (h_0, c_0))
       
        pred = self.linear(output)  # pred()
        pred = pred[:, -1, :]     
        return pred

In [ ]:
# 定义模型
# from turtle import forward
import torch.nn as nn
import torch.nn.functional as F
# 定义一个类
class CNN_BILSTM(nn.Module):
    def __init__(self,in_channels,out_channels,hidden_size,num_layers,output_size,batch_size,seq_length) -> None:
        super(CNN_BILSTM,self).__init__()
        self.in_channels=in_channels  
        self.out_channels=out_channels 
        self.hidden_size=hidden_size 
        self.num_layers=num_layers 
        self.output_size=output_size 
        self.batch_size=batch_size 
        self.seq_length=seq_length 
        self.num_directions=2 
        self.relu = nn.ReLU(inplace=True) 
        # (batch_size=64, seq_len=3, input_size=3) ---> permute(0, 2, 1)
        # (64, 3, 3)
        self.conv=nn.Sequential(
            nn.Conv1d(in_channels=self.in_channels, out_channels=self.out_channels, kernel_size=3), #shape(7,--)  ->(64,3,2)
            nn.ReLU())
        self.lstm=nn.LSTM(input_size=out_channels,hidden_size=hidden_size,num_layers=num_layers,batch_first=True,bidirectional=True)
        self.fc = nn.Linear(hidden_size*self.num_directions, output_size)
      

    def forward(self,x):
        x=x.permute(0,2,1)                            
        x=self.conv(x) 
        x=x.permute(0,2,1) 
        batch_size, seq_len = x.size()[0], x.size()[1] 
        h_0 = torch.randn(self.num_directions * self.num_layers, x.size(0), self.hidden_size).to(device)
        c_0 = torch.randn(self.num_directions * self.num_layers, x.size(0), self.hidden_size).to(device)
        # output(batch_size, seq_len, num_directions * hidden_size)
        output, _ = self.lstm(x, (h_0, c_0)) 
        # print(output.shape)
        pred = self.fc(output)   
        pred = pred[:, -1, :]  
        return pred 

In [ ]:
# 定义模型
# from turtle import forward
import torch.nn as nn
import torch.nn.functional as F
# 定义一个类
class CNN_BILSTM_Attention(nn.Module): 
    def __init__(self,in_channels,out_channels,hidden_size,num_layers,output_size,batch_size,seq_length) -> None:
        super(CNN_BILSTM_Attention,self).__init__()
        self.in_channels=in_channels    
        self.out_channels=out_channels
        self.hidden_size=hidden_size
        self.num_layers=num_layers 
        self.output_size=output_size 
        self.batch_size=batch_size 
        self.seq_length=seq_length 
        self.num_directions=2 # 单向LSTM
        self.relu = nn.ReLU(inplace=True)
        # (batch_size=64, seq_len=3, input_size=3) ---> permute(0, 2, 1)
        # (64, 3, 3)
        self.conv=nn.Sequential(
            nn.Conv1d(in_channels=self.in_channels, out_channels=self.out_channels, kernel_size=3), #shape(7,--)  ->(64,3,2)
            nn.ReLU()) 
        self.lstm=nn.LSTM(input_size=out_channels,hidden_size=hidden_size,num_layers=num_layers,batch_first=True,bidirectional=True)
        self.fc = nn.Linear(hidden_size*self.num_directions, output_size)
      
        
    def attention_net(self, lstm_output, final_state):
        lstm_output = lstm_output.permute(1, 0, 2)
        hidden = final_state.view(-1,self.num_directions*self.hidden_size,self.num_layers)   # hidden : [batch_size, n_hidden * num_directions(=2), 1(=n_layer)]
        # print(lstm_output.shape,hidden.shape)
        self.attn_weights = torch.bmm(lstm_output, hidden) # attn_weights : [batch_size, n_step]

        soft_attn_weights = torch.tanh(self.attn_weights)

        context = torch.bmm(lstm_output.transpose(1, 2), soft_attn_weights).squeeze(2)
        return context # context : [batch_size, n_hidden * num_directions(=2)]


    def forward(self,x):
        x=x.permute(0,2,1)
        x=self.conv(x)
        x=x.permute(0,2,1)
        batch_size, seq_len = x.size()[0], x.size()[1]    
        
        h_0 = torch.randn(self.num_directions * self.num_layers, x.size(0), self.hidden_size).to(device)
        c_0 = torch.randn(self.num_directions * self.num_layers, x.size(0), self.hidden_size).to(device)
        # output(batch_size, seq_len, num_directions * hidden_size)
        lstm_out, self.hidden_cell = self.lstm(x) 
        lstm_out=lstm_out.permute(1,0,2)
        # print('lstm_out',lstm_out.shape,self.hidden_cell[0].shape,self.hidden_cell[1].shape)
        attn_output = self.attention_net(lstm_out,self.hidden_cell[0])
        # print('attn_output',attn_output.shape)
        pred = self.fc(attn_output.view(-1,self.hidden_size*self.num_directions))  
        # print(pred.shape)
        # pred = pred[:, -1, :]  
        return pred

训练模型

bilstm

In [ ]:
epochs=100 
lr=0.004 
momentum=0.2 
weight_decay=6e-4  
in_channels=input_size=5 
out_channels=128 
hidden_size=64 
num_layers=1 
batch_size=30 
output_size=1 
seq_length=7 

In [ ]:
model_name='BILSTM'
model_att=BiLSTM_BIGRU(input_size, hidden_size, num_layers, output_size, batch_size).to(device)
loss_function=nn.MSELoss().to(device)
optimizer_att = torch.optim.SGD(model_att.parameters(), lr=lr,momentum=momentum, weight_decay=weight_decay)
print(model_att)


In [ ]:
value_train_dic=[] 
loss_train_dic=[] 
value_val_dic=[] 
loss_val_dic=[] 
path='model_result.txt' 

In [ ]:
import random
def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     np.random.seed(seed)
     random.seed(seed)
     torch.backends.cudnn.deterministic = True
# 设置随机数种子
setup_seed(20)

In [ ]:
from tqdm import tqdm_notebook as tqdm
import copy
def train_model(model,optimizer):
    model.train()  
    print('训练','模型') 
    min_epochs=0 
    best_model=None 
    min_val_loss=5 
    train_loss_list=[]  
    val_loss_list=[] 
    for epoch in tqdm(range(epochs)): 
        #train
        add=0
        train_loss=[]  
        val_loss=[]  
        for (seq,label) in train_loader:     
            optimizer.zero_grad()  
            seq=seq.to(device)  
            label=label.to(device)  
            y_pred = model(seq)  
            
            if(epoch==epochs-1):  
                value_train_dic.append(y_pred)  
            # print(y_pred.shape,label.shape)
            single_loss = loss_function(y_pred, label.reshape(-1,1))   
            train_loss.append(single_loss.item())    
            single_loss .backward()    
            optimizer.step()   

        loss_train_dic.append(np.mean(train_loss))   
        

        #val    
        add=0 
        t=0

        for (seq,label) in test_loader:  
            with torch.no_grad():   
                seq = seq.to(device)    
                label=label.to(device)    
                y_pred=model(seq)   
                single_loss=loss_function(y_pred,label.reshape(-1,1))    
                val_loss.append(single_loss.item())    
                
                if(epoch==epochs): 
                    value_val_dic.append(y_pred)    

        loss_val_dic.append(np.mean(val_loss))    

        print('epoch {:03d} train_loss {:.8f} val_loss {:.8f}'.format(epoch, np.mean(train_loss), np.mean(val_loss)))

        train_loss_list.append(np.mean(train_loss)) 
        val_loss_list.append(np.mean(val_loss))   
        if epoch > min_epochs and np.mean(val_loss) < min_val_loss:   
            min_val_loss = np.mean(val_loss)   
            best_model = copy.deepcopy(model)   

    state = {'models': best_model.state_dict()}   
    torch.save(state, path)  
    print('----------------------')
    return train_loss_list,val_loss_list  

In [ ]:
train_loss_list,val_loss_list=train_model(model_att,optimizer_att)

In [ ]:
#loss曲线
plt.figure(figsize=(12,4),dpi=120)  
plt.plot(range(epochs),train_loss_list,label="loss") 
plt.plot(range(epochs),val_loss_list,label="val_loss") 
plt.title("Loss Curve",fontsize=18) 
plt.xlabel("Epochs",fontsize=15)
plt.ylabel("Loss",fontsize=15)   
plt.grid(alpha=0.3) 
plt.legend()  
plt.title('bilstm')   
plt.savefig('bilstm-loss.jpg',bbox_inches = 'tight',dpi=600) 
plt.show() 

In [ ]:
model=model_att 
pred=[] 
y_true=[]  
loss=[]   
model.load_state_dict(torch.load(path)['models'])  
model.eval()   

for (seq,target) in tqdm(test_loader):   
    seq = seq.to(device)  
    target=target.to(device)  

    y_true.extend(target.cpu())  
    with torch.no_grad():  
      # model.hidden = (torch.zeros(1, 1, model.hidden_layer_size),
      #                 torch.zeros(1, 1, model.hidden_layer_size))
        y_pred=model(seq)  
        pred.extend(y_pred.cpu())  

    loss.append(loss_function(y_pred, target.reshape(-1,1))) 

y_true, pred = np.array(y_true), np.array(pred) 
y_true = y_true.reshape(-1,1)   
pred = pred.reshape(-1,1)  
print(y_true.shape,pred.shape) 
y_true=scaler2.inverse_transform(y_true).ravel() 
pred=scaler2.inverse_transform(pred).ravel()   

#保存结果
data = {'y_true': y_true,'pred': pred}
df = pd.DataFrame(data, index=y_validate_index.ravel()[:len(y_true)])

# 保存到Excel
df.to_excel('person-BiLSTM预测值和真实值.xlsx')


x = [i for i in range(len(y_true))]
plt.figure(figsize=(12, 6),dpi=120)   
# plt.grid(True)


plt.plot(y_validate_index.ravel()[:len(y_true)],y_true,color="red",label='real value')
plt.plot(y_validate_index.ravel()[:len(y_true)],pred,label='prediction')

plt.title('bilstm')   
plt.ylabel('demand')  
plt.xlabel('date')   

plt.legend(loc='upper right',fontsize=15)  
plt.savefig('bilstm-true-vs-predict.jpg',bbox_inches = 'tight',dpi=600) 

print('rmse: ',metrics.mean_squared_error(y_true, pred,squared=False),
      'mae: ',metrics.mean_absolute_error(y_true, pred),
      'r2: ',metrics.r2_score(y_true, pred)
     )
result_1=[metrics.mean_squared_error(y_true, pred,squared=False),metrics.mean_absolute_error(y_true, pred),metrics.r2_score(y_true, pred)]

bilstm-attention

In [ ]:
epochs=100
lr=0.004
momentum=0.2
weight_decay=6e-4
in_channels=input_size=5
out_channels=128
hidden_size=64
num_layers=1
batch_size=30
output_size=1
seq_length=7

In [ ]:
model_name='BILSTM-ATT'
model_att=BILSTM_BIGRU_Attention(input_size, hidden_size, num_layers, output_size, batch_size).to(device)
loss_function=nn.MSELoss().to(device)
optimizer_att = torch.optim.SGD(model_att.parameters(), lr=lr,momentum=momentum, weight_decay=weight_decay)
print(model_att)

In [ ]:
value_train_dic=[]
loss_train_dic=[]
value_val_dic=[]
loss_val_dic=[]
path='model_result.txt'

In [ ]:
import random
def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     np.random.seed(seed)
     random.seed(seed)
     torch.backends.cudnn.deterministic = True
# 设置随机数种子
setup_seed(20)

In [ ]:
import copy
def train_model(model,optimizer):
    model.train()
    print('训练','模型')
    min_epochs=0
    best_model=None
    min_val_loss=5
    train_loss_list=[]
    val_loss_list=[]
    for epoch in tqdm(range(epochs)):
        #train
        add=0
        train_loss=[]
        val_loss=[]
        for (seq,label) in train_loader:   
            optimizer.zero_grad()
            seq=seq.to(device)
            label=label.to(device)
            y_pred = model(seq)

            if(epoch==epochs-1):  #对最后一次epoch的值进行记录
                value_train_dic.append(y_pred)
            # print(y_pred.shape,label.shape)
            single_loss = loss_function(y_pred, label.reshape(-1,1))   
            train_loss.append(single_loss.item())
            single_loss .backward()
            optimizer.step()

        loss_train_dic.append(np.mean(train_loss))

        #val
        add=0 
        t=0

        for (seq,label) in test_loader:
            with torch.no_grad():
                seq = seq.to(device)
                label=label.to(device)
                y_pred=model(seq)
                single_loss=loss_function(y_pred,label.reshape(-1,1))
                val_loss.append(single_loss.item())

                if(epoch==epochs):  #对最后一次epoch的值进行记录
                    value_val_dic.append(y_pred)

        loss_val_dic.append(np.mean(val_loss))

        print('epoch {:03d} train_loss {:.8f} val_loss {:.8f}'.format(epoch, np.mean(train_loss), np.mean(val_loss)))
        train_loss_list.append(np.mean(train_loss))
        val_loss_list.append(np.mean(val_loss))
        if epoch > min_epochs and np.mean(val_loss) < min_val_loss:
            min_val_loss = np.mean(val_loss)
            best_model = copy.deepcopy(model)

    state = {'models': best_model.state_dict()}
    torch.save(state, path)
    print('----------------------')
    return train_loss_list,val_loss_list

In [ ]:
train_loss_list,val_loss_list=train_model(model_att,optimizer_att)

In [ ]:
#loss曲线
plt.figure(figsize=(12,4),dpi=120)
plt.plot(range(epochs),train_loss_list,label="loss") # Loss curve for training set
plt.plot(range(epochs),val_loss_list,label="val_loss") # Loss curve for validation set
plt.title("Loss Curve",fontsize=18)
plt.xlabel("Epochs",fontsize=15)
plt.ylabel("Loss",fontsize=15)
plt.grid(alpha=0.3)
plt.legend()
plt.title('bilstm-attention')
plt.savefig('bilstm-attention-loss.jpg',bbox_inches = 'tight',dpi=600)
plt.show()

In [ ]:
model=model_att
pred=[]
y_true=[]
loss=[]
model.load_state_dict(torch.load(path)['models'])
model.eval()

for (seq,target) in tqdm(test_loader):
    seq = seq.to(device)
    target=target.to(device)

    y_true.extend(target.cpu())
    with torch.no_grad():
      # model.hidden = (torch.zeros(1, 1, model.hidden_layer_size),
      #                 torch.zeros(1, 1, model.hidden_layer_size))
        y_pred=model(seq)
        pred.extend(y_pred.cpu())

    loss.append(loss_function(y_pred, target.reshape(-1,1)))

y_true, pred = np.array(y_true), np.array(pred)
y_true = y_true.reshape(-1,1)
pred = pred.reshape(-1,1)
print(y_true.shape,pred.shape)
y_true=scaler2.inverse_transform(y_true).ravel()
pred=scaler2.inverse_transform(pred).ravel()


#保存结果

data = {'y_true': y_true,'pred': pred}
df = pd.DataFrame(data, index=y_validate_index.ravel()[:len(y_true)])

# 保存到Excel
df.to_excel('person-BiLSTM-ATT预测值和真实值.xlsx')

x = [i for i in range(len(y_true))]
plt.figure(figsize=(12, 6),dpi=120)
# plt.grid(True)

plt.plot(y_validate_index.ravel()[:len(y_true)],y_true,color="red",label='real value')
plt.plot(y_validate_index.ravel()[:len(y_true)],pred,label='prediction')

plt.title('bilstm-attention')
plt.ylabel('demand')
plt.xlabel('date')

plt.legend(loc='upper right',fontsize=15)
plt.savefig('bilstm-attention-true-vs-predict.jpg',bbox_inches = 'tight',dpi=600)
print('rmse: ',metrics.mean_squared_error(y_true, pred,squared=False),
      'mae: ',metrics.mean_absolute_error(y_true, pred),
      'r2: ',metrics.r2_score(y_true, pred)
     )
result_2=[metrics.mean_squared_error(y_true, pred,squared=False),metrics.mean_absolute_error(y_true, pred),metrics.r2_score(y_true, pred)]

cnn-bilstm

In [ ]:
epochs=100
lr=0.004
momentum=0.2
weight_decay=6e-4
in_channels=input_size=5
out_channels=128
hidden_size=64
num_layers=1
batch_size=30
output_size=1
seq_length=7

In [ ]:
model_att=CNN_BILSTM(in_channels,out_channels, hidden_size, num_layers, output_size, batch_size,seq_length).to(device)
loss_function=nn.MSELoss().to(device)
optimizer_att = torch.optim.SGD(model_att.parameters(), lr=lr,momentum=momentum, weight_decay=weight_decay)
print(model_att)

In [ ]:
value_train_dic=[]
loss_train_dic=[]
value_val_dic=[]
loss_val_dic=[]
path='model_result.txt'

In [ ]:
import random
def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     np.random.seed(seed)
     random.seed(seed)
     torch.backends.cudnn.deterministic = True
# 设置随机数种子
setup_seed(20)

In [ ]:
import copy
def train_model(model,optimizer):
    model.train()
    print('训练','模型')
    min_epochs=0
    best_model=None
    min_val_loss=5
    train_loss_list=[]
    val_loss_list=[]
    for epoch in tqdm(range(epochs)):
        #train
        add=0
        train_loss=[]
        val_loss=[]
        for (seq,label) in train_loader:   
            optimizer.zero_grad()
            seq=seq.to(device)
            label=label.to(device)
            y_pred = model(seq)

            if(epoch==epochs-1):  
                value_train_dic.append(y_pred)
            # print(y_pred.shape,label.shape)
            single_loss = loss_function(y_pred, label.reshape(-1,1))   
            train_loss.append(single_loss.item())
            single_loss .backward()
            optimizer.step()

        loss_train_dic.append(np.mean(train_loss))

        #val
        add=0 
        t=0

        for (seq,label) in test_loader:
            with torch.no_grad():
                seq = seq.to(device)
                label=label.to(device)
                y_pred=model(seq)
                single_loss=loss_function(y_pred,label.reshape(-1,1))
                val_loss.append(single_loss.item())

                if(epoch==epochs):  #对最后一次epoch的值进行记录
                    value_val_dic.append(y_pred)

        loss_val_dic.append(np.mean(val_loss))

        print('epoch {:03d} train_loss {:.8f} val_loss {:.8f}'.format(epoch, np.mean(train_loss), np.mean(val_loss)))
        train_loss_list.append(np.mean(train_loss))
        val_loss_list.append(np.mean(val_loss))
        if epoch > min_epochs and np.mean(val_loss) < min_val_loss:
            min_val_loss = np.mean(val_loss)
            best_model = copy.deepcopy(model)

    state = {'models': best_model.state_dict()}
    torch.save(state, path)
    print('----------------------')
    return train_loss_list,val_loss_list

In [ ]:
train_loss_list,val_loss_list=train_model(model_att,optimizer_att)

In [ ]:
#loss曲线
plt.figure(figsize=(12,4),dpi=120)
plt.plot(range(epochs),train_loss_list,label="loss") # Loss curve for training set
plt.plot(range(epochs),val_loss_list,label="val_loss") # Loss curve for validation set
plt.title("Loss Curve",fontsize=18)
plt.xlabel("Epochs",fontsize=15)
plt.ylabel("Loss",fontsize=15)
plt.grid(alpha=0.3)
plt.legend()
plt.title('cnn-bilstm')
plt.savefig('cnn-bilstm-loss.jpg',bbox_inches = 'tight',dpi=600)
plt.show()

In [ ]:
model=model_att
pred=[]
y_true=[]
loss=[]
model.load_state_dict(torch.load(path)['models'])
model.eval()

for (seq,target) in tqdm(test_loader):
    seq = seq.to(device)
    target=target.to(device)

    y_true.extend(target.cpu())
    with torch.no_grad():
      # model.hidden = (torch.zeros(1, 1, model.hidden_layer_size),
      #                 torch.zeros(1, 1, model.hidden_layer_size))
        y_pred=model(seq)
        pred.extend(y_pred.cpu())

    loss.append(loss_function(y_pred, target.reshape(-1,1)))

y_true, pred = np.array(y_true), np.array(pred)
y_true = y_true.reshape(-1,1)
pred = pred.reshape(-1,1)
print(y_true.shape,pred.shape)
y_true=scaler2.inverse_transform(y_true).ravel()
pred=scaler2.inverse_transform(pred).ravel()


#保存结果
data = {'y_true': y_true,'pred': pred}
df = pd.DataFrame(data, index=y_validate_index.ravel()[:len(y_true)])

# 保存到Excel
df.to_excel('person-CNN-BiLSTM预测值和真实值.xlsx')

x = [i for i in range(len(y_true))]
plt.figure(figsize=(12, 6),dpi=120)
# plt.grid(True)

plt.plot(y_validate_index.ravel()[:len(y_true)],y_true,color="red",label='real value')
plt.plot(y_validate_index.ravel()[:len(y_true)],pred,label='prediction')

plt.title('cnn-bilstm')
plt.ylabel('demand')
plt.xlabel('date')

plt.legend(loc='upper right',fontsize=15)
plt.savefig('cnn-bilstm-true-vs-predict.jpg',bbox_inches = 'tight',dpi=600)
print('rmse: ',metrics.mean_squared_error(y_true, pred,squared=False),
      'mae: ',metrics.mean_absolute_error(y_true, pred),
      'r2: ',metrics.r2_score(y_true, pred)
     )
result_5=[metrics.mean_squared_error(y_true, pred,squared=False),metrics.mean_absolute_error(y_true, pred),metrics.r2_score(y_true, pred)]

cnn-bilstm-attention

In [ ]:
epochs=100
lr=0.004
momentum=0.2
weight_decay=6e-4
in_channels=input_size=5
out_channels=128
hidden_size=64
num_layers=1
batch_size=30
output_size=1
seq_length=7

In [ ]:
model_att=CNN_BILSTM_Attention(in_channels,out_channels, hidden_size, num_layers, output_size, batch_size,seq_length).to(device)
loss_function=nn.MSELoss().to(device)
optimizer_att = torch.optim.SGD(model_att.parameters(), lr=lr,momentum=momentum, weight_decay=weight_decay)
print(model_att)

In [ ]:
value_train_dic=[]
loss_train_dic=[]
value_val_dic=[]
loss_val_dic=[]
path='model_result.txt'

In [ ]:
import random
def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     np.random.seed(seed)
     random.seed(seed)
     torch.backends.cudnn.deterministic = True
# 设置随机数种子
setup_seed(20)

In [ ]:
import copy
def train_model(model,optimizer):
    model.train()
    print('训练','模型')
    min_epochs=0
    best_model=None
    min_val_loss=5
    train_loss_list=[]
    val_loss_list=[]
    for epoch in tqdm(range(epochs)):
        #train
        add=0
        train_loss=[]
        val_loss=[]
        for (seq,label) in train_loader:   
            optimizer.zero_grad()
            seq=seq.to(device)
            label=label.to(device)
            y_pred = model(seq)

            if(epoch==epochs-1):  #对最后一次epoch的值进行记录
                value_train_dic.append(y_pred)
            # print(y_pred.shape,label.shape)
            single_loss = loss_function(y_pred, label.reshape(-1,1))   
            train_loss.append(single_loss.item())
            single_loss .backward()
            optimizer.step()

        loss_train_dic.append(np.mean(train_loss))

        #val
        add=0 
        t=0

        for (seq,label) in test_loader:
            with torch.no_grad():
                seq = seq.to(device)
                label=label.to(device)
                y_pred=model(seq)
                single_loss=loss_function(y_pred,label.reshape(-1,1))
                val_loss.append(single_loss.item())

                if(epoch==epochs):  
                    value_val_dic.append(y_pred)

        loss_val_dic.append(np.mean(val_loss))

        print('epoch {:03d} train_loss {:.8f} val_loss {:.8f}'.format(epoch, np.mean(train_loss), np.mean(val_loss)))
        train_loss_list.append(np.mean(train_loss))
        val_loss_list.append(np.mean(val_loss))
        if epoch > min_epochs and np.mean(val_loss) < min_val_loss:
            min_val_loss = np.mean(val_loss)
            best_model = copy.deepcopy(model)

    state = {'models': best_model.state_dict()}
    torch.save(state, path)
    print('----------------------')
    return train_loss_list,val_loss_list

In [ ]:
train_loss_list,val_loss_list=train_model(model_att,optimizer_att)

In [ ]:
#loss曲线
plt.figure(figsize=(12,4),dpi=120)
plt.plot(range(epochs),train_loss_list,label="loss") # Loss curve for training set
plt.plot(range(epochs),val_loss_list,label="val_loss") # Loss curve for validation set
plt.title("Loss Curve",fontsize=18)
plt.xlabel("Epochs",fontsize=15)
plt.ylabel("Loss",fontsize=15)
plt.grid(alpha=0.3)
plt.legend()
plt.title('cnn-bilstm-attention')
plt.savefig('cnn-bilstm-attention-loss.jpg',bbox_inches = 'tight',dpi=600)
plt.show()

In [ ]:
model=model_att
pred=[]
y_true=[]
loss=[]
model.load_state_dict(torch.load(path)['models'])
model.eval()

for (seq,target) in tqdm(test_loader):
    seq = seq.to(device)
    target=target.to(device)

    y_true.extend(target.cpu())
    with torch.no_grad():
      # model.hidden = (torch.zeros(1, 1, model.hidden_layer_size),
      #                 torch.zeros(1, 1, model.hidden_layer_size))
        y_pred=model(seq)
        pred.extend(y_pred.cpu())

    loss.append(loss_function(y_pred, target.reshape(-1,1)))

y_true, pred = np.array(y_true), np.array(pred)
y_true = y_true.reshape(-1,1)
pred = pred.reshape(-1,1)
print(y_true.shape,pred.shape)
y_true=scaler2.inverse_transform(y_true).ravel()
pred=scaler2.inverse_transform(pred).ravel()


#保存结果

data = {'y_true': y_true,'pred': pred}
df = pd.DataFrame(data, index=y_validate_index.ravel()[:len(y_true)])

# 保存到Excel
df.to_excel('person-CNN-BiLSTM-ATT预测值和真实值.xlsx')

x = [i for i in range(len(y_true))]
plt.figure(figsize=(12, 6),dpi=120)
# plt.grid(True)

plt.plot(y_validate_index.ravel()[:len(y_true)],y_true,color="red",label='real value')
plt.plot(y_validate_index.ravel()[:len(y_true)],pred,label='prediction')

plt.title('cnn-bilstm-attention')
plt.ylabel('demand')
plt.xlabel('date')

plt.legend(loc='upper right',fontsize=15)
plt.savefig('cnn-bilstm-attention-true-vs-predict.jpg',bbox_inches = 'tight',dpi=600)
print('rmse: ',metrics.mean_squared_error(y_true, pred,squared=False),
      'mae: ',metrics.mean_absolute_error(y_true, pred),
      'r2: ',metrics.r2_score(y_true, pred)
     )
result_6=[metrics.mean_squared_error(y_true, pred,squared=False),metrics.mean_absolute_error(y_true, pred),metrics.r2_score(y_true, pred)]

模型对比


In [ ]:
result=pd.DataFrame([result_1,result_2,result_3,result_4,result_5,result_6],
             columns=['test_rmse','test_mae','test_r2'],
             index=['bilstm','bilstm-attention','bigru','bigru-attention','cnn-bilstm','cnn-bilstm-attention']
            )
result.to_excel('person前6特征的模型对比.xlsx')
result

shap

In [ ]:
torch.backends.cudnn.enabled = False  
model=CNN_BILSTM(in_channels,out_channels, hidden_size, num_layers, output_size, batch_size,seq_length).to(device)
model.load_state_dict(torch.load('model_result_cnn_bilstm.txt',map_location=torch.device('cpu'))['models'])
model.eval()

In [ ]:
import shap
data_tensor = torch.tensor(x_test[:100]).to(device)
data_tensor = data_tensor.clone().detach().requires_grad_(True)
#shap_values = explainer(x)
explainer = shap.DeepExplainer(model, data_tensor)
shap_values = explainer.shap_values(data_tensor)

In [ ]:
cols=df.columns

In [ ]:
x_test1=pd.DataFrame(x_test[:100:,0,:],columns=cols)

In [ ]:
from shap import LinearExplainer, KernelExplainer, Explanation
shap_values1 = Explanation(shap_values[:,0,:],explainer.expected_value, data=x_test1,feature_names=cols)
shap_values1.base_values=np.array([shap_values1.base_values[0] for i in range(100)])

In [ ]:
import warnings
warnings.filterwarnings("ignore")
shap.summary_plot(shap_values[:,0,:], x_test1, show=False)
plt.savefig('summary_plot1.jpg',dpi=600, bbox_inches = 'tight')
plt.show()

In [ ]:
# summary_plot 
shap.summary_plot(shap_values[:,0,:], x_test1,plot_size=(12,6),plot_type="bar",show=False)
plt.savefig('summary_plot2.jpg',dpi=600, bbox_inches = 'tight')
plt.show()